# Function to test in the early phrese

From a sequence, chose casually n positions and from these poritions takes m foward bases, creating  in this way a read
starting at position n, with lenght m. The number of reads are: $ (len(seq) / lenght_reads) \times coverage  $. Can creates holes.

In [23]:
from Bio import pairwise2
from Bio.Seq import Seq
from math import modf
from inspyred import swarm
from inspyred import ec
from inspyred.ec import selectors
from collections import deque
import numpy as np
import itertools
import random


In [55]:


ref = """ATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAGCCCCCTCTGAGTCAGGAAACATTTTCAGACCTATGGAAACTACTTCCTGAAAACAACGTTCTGTCCCCCTTGCCGTCCCAAGCAATGGATGATTTGATGCTGTCCCCGGACGATATTGAACAATGGTTCACTGAAGACCCAGGTCCAGATGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCCCGTGGCCCCTGCACCAGCAGCTCCTACACCGGCGGCCCCTGCACCAGCCCCCTCCTGGCCCCTGTCATCTTCTGTCCCTTCCCAGAAAACCTACCAGGGCAGCTACGGTTTCCGTCTGGGCTTCTTGCATTCTGGGACAGCCAAGTCTGTGACTTGCACGTACTCCCCTGCCCTCAACAAGATGTTTTGCCAACTGGCCAAGACCTGCCCTGTGCAGCTGTGGGTTGATTCCACACCCCCGCCCGGCACCCGCGTCCGCGCCATGGCCATCTACAAGCAGTCACAGCACATGACGGAGGTTGTGAGGCGCTGCCCCCACCATGAGCGCTGCTCAGATAGCGATGGTCTGGCCCCTCCTCAGCATCTTATCCGAGTGGAAGGAAATTTGCGTGTGGAGTATTTGGATGACAGAAACACTTTTCGACATAGTGTGGTGGTGCCCTATGAGCCGCCTGAGGTTGGCTCTGACTGTACCACCATCCACTACAACTACATGTGTAACAGTTCCTGCATGGGCGGCATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCAGTGGTAATCTACTGGGACGGAACAGCTTTGAGGTGCGTGTTTGTGCCTGTCCTGGGAGAGACCGGCGCACAGAGGAAGAGAATCTCCGCAAGAAAGGGGAGCCTCACCACGAGCTGCCCCCAGGGAGCACTAAGCGAGCACTGCCCAACAACACCAGCTCCTCTCCCCAGCCAAAGAAGAAACCACTGGATGGAGAATATTTCACCCTTCAGATCCGTGGGCGTGAGCGCTTCGAGATGTTCCGAGAGCTGAATGAGGCCTTGGAACTCAAGGATGCCCAGGCTGGGAAGGAGCCAGGGGGGAGCAGGGCTCACTCCAGCCACCTGAAGTCCAAAAAGGGTCAGTCTACCTCCCGCCATAAAAAACTCATGTTCAAGACAGAAGGGCCTGACTCAGACTGA"""
def comstum_reads(seq: str, length_reads = 10, coverage = 5) -> list:

    number_of_reads = int(len(seq)/length_reads) * coverage
    starting_pos = random.sample(range(0, len(seq)), number_of_reads)
    reads = []

    for num in starting_pos:
        over = num + length_reads
        if over > len(seq):
            reads.append(seq[-length_reads:])
        else:
            reads.append(seq[num:num+length_reads])

    return reads

reads = comstum_reads(ref, length_reads=20)

In [26]:
def eval_allign(reads:list, par = [2, -5, -22, -5]):
    """Funtion that evaulate the alignment

    reads: list of DNA sequences

    par: list of parameters to performe the alignment
    es (the examples represent the defoult parameters):
    match_score = 2,
    mismatch_penalty = -5,
    gap_open_penalty = -22,
    gap_extension_penalty = -5

    output:
    matrix with the weigts (distances) between the reads (nodes)
    """
    length = len(reads)
    # initialization of the matrices
    weigth_matrix = np.zeros((length, length))

    # The score of the allingment of read[1] to read[2] is the same of the opposite (read[2] to read[1])
    # So when the function found the diretionality of the allignment put the score in rigth spot and a 0 in the wrong one.
    visited = deque([j for j in range(length)])
    
    for i in range(length):

        for j in visited:

            if i == j:
                # the diagonal of the matrix has 0 score because we are allinging the same sequence
                continue
            else:
                allignment = pairwise2.align.localms(Seq(reads[i]), Seq(reads[j]), par[0], par[1], par[2], par[3])[0]
                start = allignment[3]
                end = allignment[4]
                # return object [seqA, seqB, score, start(inc), end(ex)]

                if allignment[0][0] == "-":
                    # This means that the first reads in input has a gap at the beginning of the allignment.
                    # Therefore the first reads in input (i) is downstream,
                    # so I add the score in the matrix but in the position (j,i) instead of (i,j) where there is a 0
                    weigth_matrix[j][i] = allignment[2]
                    weigth_matrix[i][j] = float(f"{start}.{end-1}")

                else:
                    # In the opposite case, where the i read is upstream (i,j) has the score, while (j,i) has a 0                    #
                    weigth_matrix[i][j] = allignment[2]
                    weigth_matrix[j][i] = float(f"{start}.{end-1}")

                    
        visited.popleft()

    return weigth_matrix

# if modf(weights[i,j])[0] > 0

In [ ]:

def reconstruct_sequence(path:list, reads:list, positions:list) -> str:
    """Rebluild from the list of reds, the path and the matrix with the scores the allignment

    path:list of tuple with edges
    reads: list of the reads
    positions: is the weigth matrix, but will be considered only the number linked with the base overlapping

    output: a string with the sequece    
    """
    tot_seq = ""
    cnt = 0 
    # cnt is needed to know when the func is in the first and last node/read

    for i,j in path:
        # Here i,j represent the edge of the graph, to retrive not the score but the alignment
        # the function needs the opposite position where there are these informations matrix[j][i]
        # something like 12.22, 12 is the strating base 22 is the ending base of the overlapping both included

        num = str(positions[j][i]).split(".")
        start = int(num[0])  # included
        end = int(num[1])   # included

        if cnt == 0:
            tot_seq += reads[i][:end]   # The func takes alaways reads[i] because the graph is direct
            cnt += 1
            
        elif cnt == len(path)-1:
            tot_seq += reads[j][start:]
            
        else:
            tot_seq += reads[i][start:end]
            cnt += 1
            
    return tot_seq

In [57]:
# pairwise2.align.globalm(ref,"ATTGACAC" ,3,-5)[0]
# help(pairwise2.align.globalmx)

Alignment(seqA='ATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAGCCCCCTCTGAGTCAGGAAACATTTTCAGACCTATGGAAACTACTTCCTGAAAACAACGTTCTGTCCCCCTTGCCGTCCCAAGCAATGGATGATTTGATGCTGTCCCCGGACGATATTGAACAATGGTTCACTGAAGACCCAGGTCCAGATGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCCCGTGGCCCCTGCACCAGCAGCTCCTACACCGGCGGCCCCTGCACCAGCCCCCTCCTGGCCCCTGTCATCTTCTGTCCCTTCCCAGAAAACCTACCAGGGCAGCTACGGTTTCCGTCTGGGCTTCTTGCATTCTGGGACAGCCAAGTCTGTGACTTGCACGTACTCCCCTGCCCTCAACAAGATGTTTTGCCAACTGGCCAAGACCTGCCCTGTGCAGCTGTGGGTTGATTCCACACCCCCGCCCGGCACCCGCGTCCGCGCCATGGCCATCTACAAGCAGTCACAGCACATGACGGAGGTTGTGAGGCGCTGCCCCCACCATGAGCGCTGCTCAGATAGCGATGGTCTGGCCCCTCCTCAGCATCTTATCCGAGTGGAAGGAAATTTGCGTGTGGAGTATTTGGATGACAGAAACACTTTTCGACATAGTGTGGTGGTGCCCTATGAGCCGCCTGAGGTTGGCTCTGACTGTACCACCATCCACTACAACTACATGTGTAACAGTTCCTGCATGGGCGGCATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCAGTGGTAATCTACTGGGACGGAACAGCTTTGAGGTGCGTGTTTGTGCCTGTCCTGGGAGAGACCGGCGCACAGAGGAAGAGAATCTCCGCAAGAAAGGGGAGCCTCACCACGAGCTGCCCCCAGGGAGCACTAAGCGAGCACTGCCCAACAACACCAGCTCCTCTCCCCAGCCAAAGAAGAAACCACTGGATGGAGAATATTTC

In [ ]:

class Assembly_problem():

    def __init__(self, weights):
        self.weights = weights # matrix of wiegths
        self.components = [swarm.TrailComponent((i, j), value=(1 / weights[i][j])) for i, j in itertools.permutations(range(len(weights)), 2) if modf(weights[i,j])[0] > 0]
        self.bias = 0.5
        self.bounder = ec.DiscreteBounder([i for i in range(len(weights))])
        self.maximize = True

    def constructor(self, random, args):
        """Return a candidate solution for an ant colony optimization."""
        candidate = []
        while len(candidate) < len(self.weights) - 1:
            # Find feasible components
            feasible_components = []
            if len(candidate) == 0:
                feasible_components = self.components
            elif len(candidate) == len(self.weights) - 1:
                first = candidate[0]
                last = candidate[-1]
                feasible_components = [c for c in self.components if c.element[0] == last.element[1] and c.element[1] == first.element[0]]
            else:
                last = candidate[-1]
                already_visited = [c.element[0] for c in candidate]
                already_visited.extend([c.element[1] for c in candidate])
                already_visited = set(already_visited)
                feasible_components = [c for c in self.components if c.element[0] == last.element[1] and c.element[1] not in already_visited]
            if len(feasible_components) == 0:
                candidate = []
            else:
                # Choose a feasible component
                if random.random() <= self.bias:
                    next_component = max(feasible_components)
                else:
                    next_component = selectors.fitness_proportionate_selection(random, feasible_components, {'num_selected': 1})[0]
                candidate.append(next_component)
        return candidate

    def evaluator(self, candidates, args):
        """Return the fitness values for the given candidates."""
        fitness = []
        for candidate in candidates:
            total = 0
            for c in candidate:
                total += self.weights[c.element[0]][c.element[1]]
            last = (candidate[-1].element[1], candidate[0].element[0])
            total += self.weights[last[0]][last[1]]
            fitness.append(1 / total)

In [5]:
import inspyred
from utils.utils_07.exercise_1 import *
from utils.utils_07.plot_utils import *

import collections
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

# common parameters
pop_size = 100
max_generations = 50
seed = 10
prng = Random(seed)
display = True
# ACS specific parameters
evaporation_rate = 0.4
learning_rate = 0.1

args = {}
args["fig_title"] = "ACS"

# run ACS
problem = Assembly_problem(eval_allign(reads))
ac = inspyred.swarm.ACS(prng, problem.components)
ac.observer = [plot_observer]
ac.terminator = inspyred.ec.terminators.generation_termination

# print("-----------")
# print(problem.weights[1:5,1:5])
# print("-----------")
# if __name__ == "main":
#     print(problem.constructor)
# print("-----------")
# if __name__ == "main":
#     print(problem.evaluator)

# final_pop = ac.evolve(generator=problem.constructor, 
#                       evaluator=problem.evaluator, 
#                       bounder=problem.bounder,
#                       maximize=problem.maximize, 
#                       pop_size=pop_size,
#                       max_generations=max_generations,
#                       evaporation_rate=evaporation_rate,
#                       learning_rate=learning_rate,**args)
# best_ACS = max(ac.archive)



ValueError: setting an array element with a sequence.